# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline


In [19]:
df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [40]:
df['count'] = 1

f1 = 'group'
f2 = 'action'
f1c1 = 'control'
f1c2 = 'experiment'


control = df[df[f1] == f1c1].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df[f1] == f1c2].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [52]:
print('Control group size and clicks:')
print('Size:', len(control))
print('Clicks:', control.click.sum())
print('Click Rate:', control.click.mean())

Control group size and clicks:
Size: 3332
Clicks: 932.0
Click Rate: 0.2797118847539016


In [51]:
print('Experimental group size and clicks:')
print('Size:', len(experiment))
print('Clicks:', experiment.click.sum())
print('Click Rate:', experiment.click.mean())

Experimental group size and clicks:
Size: 2996
Clicks: 928.0
Click Rate: 0.3097463284379172


In [31]:
df.loc[(df['group'] == 'experiment') & (df['group'] == 'control')]['id']

Series([], Name: id, dtype: int64)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [61]:
#Your code here
t, p = stats.ttest_ind(control.click, experiment.click) 

p = p / 2

print(p)

0.0044125494574791466


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [62]:
#Your code here
exp_expected_clicks = len(experiment) * control.click.mean()
exp_expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [71]:
#Your code here
n = len(experiment)
p = control.click.mean()
var = n * (p*(1-p))
std = np.sqrt(var)

std

num_of_sd = (experiment.click.sum() - exp_expected_clicks) / std
num_of_sd

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [73]:
#Your code here
p_val = stats.norm.sf(3.66) 
p_val

0.00012610762413848634

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: While our second p-value is less than the original one, both would lead to the rejection of our null hypothesis. We can confidently say that the experiemental group clicks at a significantly higher rate than that of the control group.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.